# Лабораторная работа №3

## Подготовка входных данных

Импортируем необходимые библиотеки.

In [98]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [101]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [103]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [107]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

Отделяем таргетный столбец RainTomorrow от остального датасета.

In [110]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [113]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

*Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Напишем функцию, которая преобразует непрерывный результат регрессии в бинарную классификацию. Будет использоваться дальше.

In [121]:
def regression_to_classification(x):
    if x > 0.5:
        return 1
    else:
        return 0

### Линейная регрессия

Импортируем класс LinearRegression для классификации датасета методом линейной регрессии.

In [125]:
from sklearn.linear_model import LinearRegression

Создаём объект класса LinearRegression, реализующий метод логистической регрессии.

In [128]:
linear_model = LinearRegression()

Обучаем модель на тренировочных данных.

In [131]:
linear_model.fit(X_train, y_train)

LinearRegression()

Получаем предсказания линейной модели и преобразуем предсказания в бинарную классификацию.

In [134]:
predicted = pd.DataFrame(linear_model.predict(X_test),columns=['RainTomorrow'])
predicted = predicted['RainTomorrow'].apply(lambda x: regression_to_classification(x))

In [136]:
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Оценка:
               precision    recall  f1-score   support

       False    0.86175   0.95717   0.90696      9130
        True    0.74428   0.44803   0.55935      2540

    accuracy                        0.84636     11670
   macro avg    0.80301   0.70260   0.73315     11670
weighted avg    0.83618   0.84636   0.83130     11670



### Логическая регрессия

Импортируем класс LogisticRegression для классификации датасета методом логической регрессии.

In [140]:
from sklearn.linear_model import LogisticRegression

Создаём объект класса LogisticRegression, реализующий метод логистической регрессии.

In [143]:
logistic_model = LogisticRegression()

Обучаем модель на тренировочных данных.

In [146]:
logistic_model.fit(X_train, y_train)

LogisticRegression()

Получаем предсказания логистической модели и преобразуем предсказания в бинарную классификацию.

In [149]:
predicted = pd.DataFrame(logistic_model.predict(X_test), columns=['RainTomorrow'])
predicted = predicted['RainTomorrow'].apply(lambda x: regression_to_classification(x))

In [151]:
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Оценка:
               precision    recall  f1-score   support

       False    0.87079   0.94633   0.90699      9130
        True    0.71968   0.49528   0.58675      2540

    accuracy                        0.84816     11670
   macro avg    0.79524   0.72080   0.74687     11670
weighted avg    0.83790   0.84816   0.83729     11670

